## Consider the Convolutional Neural Network and answer the following questions.
Input shape: 32x32x3
10 5x5 filters, stride 1, pad 2

(1) What is the output volume size? 32 * 32 * 10

(2) What is the number of parameters in this layer? (Each filter has one bias) 5 * 5 * 10


# LeNet-5 FashionMNIST Classification
Implement
*   Set hyper parameters
*   Call dataset from torchvision & make dataloader
*   Build a LeNet-5
*   Instantiate LeNet & define loss function and optimizer
*   Training
*   Test







In [1]:
import torch
import torch.nn as nn
import torch.nn.init
import torch.functional as F
from torch.nn.modules.activation import ReLU
from torch.nn.modules.pooling import MaxPool2d
import torchvision
from torchvision import transforms
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

# Set seed
torch.manual_seed(777)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set hyper parameters 


In [2]:
num_epochs = 10
batch_size = 64
learning_rate = 1e-3

# Call dataset from torchvision & make dataloader

In [3]:
train_datasets = torchvision.datasets.FashionMNIST(root = "data", train = True, download = True, transform = transforms.ToTensor())
test_datasets = torchvision.datasets.FashionMNIST(root = "data", train = False, download = True, transform = transforms.ToTensor())

train_loader = DataLoader(train_datasets, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader = DataLoader(test_datasets, batch_size = batch_size, shuffle = True, drop_last = True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Build a LeNet-5

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # Input (?, 32, 32, 1) -> (?, 14, 14, 6)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size = 5, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)            
        )

        # Input (?, 14, 14, 6) -> (?, 5, 5 , 16)
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size = 6, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        self.fc1 = nn.Linear(5 * 5 * 16, 120, bias = True)
        self.fc2 = nn.Linear(120, 84, bias = True)
        self.fc3 = nn.Linear(84, 10, bias = True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

# Instantiate LeNet & define loss function and optimizer


In [5]:
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Training

In [6]:
total_batch = len(train_loader)

for epoch in range(num_epochs):
    avg_cost = 0

    for test in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X, Y = test
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch:    1] cost = 0.60104084
[Epoch:    2] cost = 0.391909599
[Epoch:    3] cost = 0.338237733
[Epoch:    4] cost = 0.306554049
[Epoch:    5] cost = 0.286485434
[Epoch:    6] cost = 0.274062276
[Epoch:    7] cost = 0.262436539
[Epoch:    8] cost = 0.255482316
[Epoch:    9] cost = 0.247794136
[Epoch:   10] cost = 0.241150931


# Test

In [24]:
correct = 0
total = len(test_loader)

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels=labels.to(device)

        # 학습시킨 model에 test image를 통과시켜 출력 계산
        outputs = model(images)
        # 가장 높은 값을 가지는 class를 정답으로 선택
        predicted = torch.argmax(outputs.data, dim = 1)
        correct += (predicted == labels).float().mean().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 88.59 %
